In [1]:
import pandas as pd
import numpy as np

In [2]:
from model_utils.utils_s2 import Predictor

In [3]:
import dgutils.pandas as dgp

In [4]:
from model_utils.util_global_struct import add_bb_bottom_left, compatible_counts, filter_non_standard_stem, validate_global_struct, LocalStructureBb, OneStepChain

In [5]:
predictor = Predictor('../2020_12_08/result/synthetic_s2_training_2/model_ckpt_ep_12.pth')

In [6]:
# # toy example?


# '''
#    bb_x  bb_y  siz_x  siz_y bb_type  n_proposal  prob_median  n_proposal_norm
# 0     4     9      6      6   hloop          15     0.930053         0.416667
# 1     1    12      4      4    stem          16     0.925148         1.000000
# 2    41    50     10     10   hloop          45     0.992122         0.450000
# 3    38    53      4      4    stem          16     0.994010         1.000000
# 4    36    55      3      3   iloop           9     0.995822         1.000000
# 5    31    60      6      6    stem          36     0.978096         1.000000
# '''


# stems = pd.DataFrame([{'bb_x': 1, 'bb_y': 12, 'siz_x': 4, 'siz_y': 4, 'prob_median': 0.93, 'n_proposal_norm': 1.0},
#                      {'bb_x': 38, 'bb_y': 53, 'siz_x': 4, 'siz_y': 4, 'prob_median': 0.99, 'n_proposal_norm': 1.0},
#                      {'bb_x': 31, 'bb_y': 60, 'siz_x': 6, 'siz_y': 6, 'prob_median': 0.98, 'n_proposal_norm': 1.0}])

# iloops = pd.DataFrame([{'bb_x': 36, 'bb_y': 55, 'siz_x': 3, 'siz_y': 3, 'prob_median': 0.99, 'n_proposal_norm': 1.0}])

# hloops = pd.DataFrame([{'bb_x': 4, 'bb_y': 9, 'siz_x': 6, 'siz_y': 6, 'prob_median': 0.93, 'n_proposal_norm': 0.83},
#                       {'bb_x': 41, 'bb_y': 50, 'siz_x': 10, 'siz_y': 10, 'prob_median': 0.99, 'n_proposal_norm': 0.9}])



In [7]:
# predictor.predict(stems, iloops, hloops)

In [8]:
def summarize_df(df, hloop=False):
    # calculate median prob and n_proposal_norm
    
    def _tmp(siz_x, siz_y, prob):
        prob_median = np.median(prob)
        n_proposal_norm = len(prob)/float(siz_x * siz_y)
        if hloop:
            n_proposal_norm = 2 * n_proposal_norm
        return prob_median, n_proposal_norm
        
    df = dgp.add_columns(df, ['prob_median', 'n_proposal_norm'],
                        ['siz_x', 'siz_y', 'prob'], _tmp)
    # subset columns
    df = df[['bb_x', 'bb_y', 'siz_x', 'siz_y', 'prob_median', 'n_proposal_norm']]
    return df

In [9]:
# synthetic
# TODO

# rfam
df = pd.read_pickle('../2020_11_24/data/rfam151_s1_pruned.pkl.gz')

In [10]:
data = df.iloc[4]
# data = df.iloc[6]

# data = df.iloc[26]

In [11]:
data

len                                                              72
one_idx           ([0, 1, 2, 3, 4, 5, 6, 9, 10, 11, 12, 25, 26, ...
seq               GGGCCCAUAGCUCAGUGGUAGAGUGCCUCCUUUGCAAGGAGGAUGC...
seq_id                                                    RF00005_A
bounding_boxes    [((0, 64), (7, 7), stem), ((9, 20), (4, 4), st...
bb_stem           [{'bb_x': 0, 'bb_y': 12, 'siz_x': 4, 'siz_y': ...
bb_iloop          [{'bb_x': 3, 'bb_y': 24, 'siz_x': 7, 'siz_y': ...
bb_hloop          [{'bb_x': 3, 'bb_y': 9, 'siz_x': 7, 'siz_y': 7...
df_target         [{'bb_x': 0, 'bb_y': 70, 'siz_x': 7, 'siz_y': ...
n_bb_found                                                        4
Name: 4, dtype: object

In [12]:
pd.DataFrame(data['df_target'])

,bb_x,bb_y,siz_x,siz_y,bb_type
0,0,70,7,7,stem
1,9,23,4,4,stem
2,25,41,5,5,stem
3,47,63,5,5,stem
4,12,20,9,9,hloop
5,29,37,9,9,hloop
6,51,59,9,9,hloop


In [13]:
# FIXME handle cases where some are None
df_stem = summarize_df(pd.DataFrame(data['bb_stem']))
df_iloop = summarize_df(pd.DataFrame(data['bb_iloop']))
df_hloop = summarize_df(pd.DataFrame(data['bb_hloop']), hloop=True)
# add bottom left coord
df_stem = add_bb_bottom_left(df_stem)
df_iloop = add_bb_bottom_left(df_iloop)
df_hloop = add_bb_bottom_left(df_hloop)

In [14]:
# for each iloop, check:
# how many compatible outer stems (stem.bottom_left == iloop.top_right)
df_iloop_cleanup = compatible_counts(df_iloop, df_stem, col1=['bb_x', 'bb_y'], col2=['bl_x', 'bl_y'], out_name='num_compatible_stem_outer')
# how many compatible inner stems (stem.top_right == iloop.bottom_left)
df_iloop_cleanup = compatible_counts(df_iloop_cleanup, df_stem, col1=['bl_x', 'bl_y'], col2=['bb_x', 'bb_y'], out_name='num_compatible_stem_inner')
# drop those rows without compatible stems on both ends
df_iloop_cleanup = df_iloop_cleanup[(df_iloop_cleanup['num_compatible_stem_inner'] > 0) & (df_iloop_cleanup['num_compatible_stem_outer'] > 0)]

In [15]:
# for each hloop, check:
# how many compatible outer stems (stem.bottom_left == iloop.top_right)
df_hloop_cleanup = compatible_counts(df_hloop, df_stem, col1=['bb_x', 'bb_y'], col2=['bl_x', 'bl_y'],
                                     out_name='num_compatible_stem_outer')
# drop those rows without compatible stem
df_hloop_cleanup = df_hloop_cleanup[df_hloop_cleanup['num_compatible_stem_outer'] > 0]
# drop those not symmetric & across diagonal
df_hloop_cleanup = df_hloop_cleanup[(df_hloop_cleanup['bb_x'] == df_hloop_cleanup['bl_y']) & (
        df_hloop_cleanup['bb_y'] == df_hloop_cleanup['bl_x']) & (
                                            df_hloop_cleanup['siz_x'] == df_hloop_cleanup['siz_y'])]

In [16]:
print(len(df_stem), len(df_iloop_cleanup), len(df_hloop_cleanup))

15 7 6


In [17]:

# add IDs
df_stem['id_bb'] = [f'stem_{idx}' for idx in range(len(df_stem))]
df_iloop_cleanup['id_bb'] = [f'iloop_{idx}' for idx in range(len(df_iloop_cleanup))]
df_hloop_cleanup['id_bb'] = [f'hloop_{idx}' for idx in range(len(df_hloop_cleanup))]

# bb objects
stems = [LocalStructureBb(row['bb_x'], row['bb_y'], row['siz_x'], row['siz_y'], row['id_bb'], 'stem') for
         _, row in df_stem.iterrows()]
iloops = [LocalStructureBb(row['bb_x'], row['bb_y'], row['siz_x'], row['siz_y'], row['id_bb'], 'iloop') for
          idx, (_, row) in enumerate(df_iloop_cleanup.iterrows())]
hloops = [LocalStructureBb(row['bb_x'], row['bb_y'], row['siz_x'], row['siz_y'], row['id_bb'], 'hloop') for
          idx, (_, row) in enumerate(df_hloop_cleanup.iterrows())]

In [18]:
# prep for whitelist
# find next compatible, start with iloop
iloop_os_chain = []
for iloop in iloops:
    iloop_os = OneStepChain(iloop)
    for stem in stems:
        if iloop.share_top_right_corner(stem):
            iloop_os.add_next_bb(stem)
    iloop_os_chain.append(iloop_os)

# find next compatible, start with stem
stem_os_chain = []
for stem in stems:
    stem_os = OneStepChain(stem)
    for iloop in iloops:
        if stem.share_top_right_corner(iloop):
            stem_os.add_next_bb(iloop)
    stem_os_chain.append(stem_os)

# find next compatible, start with hloop
hloop_os_chain = []
for hloop in hloops:
    hloop_os = OneStepChain(hloop)
    for stem in stems:
        if hloop.share_top_right_corner(stem):
            hloop_os.add_next_bb(stem)
    hloop_os_chain.append(hloop_os)

In [19]:
# prep for blacklist
# all pairwise compatibility of stems
distances = np.zeros((len(stems), len(stems)), dtype=object)
for i in range(len(stems)):
    for j in range(len(stems)):
        d = stems[i].bp_conflict(stems[j])
        distances[i, j] = d
        distances[j, i] = d
stem_ids = [x.id for x in stems]
df_stem_conflict = pd.DataFrame(distances, index=stem_ids, columns=stem_ids)

In [20]:
# set whitelist and blacklist

# stem: only blacklist
df_stem = dgp.add_column(df_stem, 'blacklist', ['id_bb'], 
                         lambda x: df_stem_conflict[x][df_stem_conflict[x]].index.tolist())

# iloop: whitelist x 2
# note that since we've pruned bb in the begining, both whitelists should be non-empty
df_iloop_cleanup['whitelist1'] = None
df_iloop_cleanup['whitelist2'] = None
# list 1: top right corner connect to stem
for x in iloop_os_chain:
    id_iloop = x.bb.id
    if x.next_bb is not None:
        tr_stem_ids = [y.id for y in x.next_bb]
        # setting cell value to be a list - hacky way!
        df_iloop_cleanup.loc[df_iloop_cleanup['id_bb'] == id_iloop, 'whitelist1'] = df_iloop_cleanup.loc[df_iloop_cleanup['id_bb'] == id_iloop, 'whitelist1'].apply(lambda x: tr_stem_ids)

# list 2: bottom left corner connect to stem
# doing a bit hack here since we're tracing from stems
tmp = {x: [] for x in df_iloop_cleanup['id_bb'].tolist()}
for x in stem_os_chain:
    id_stem = x.bb.id
    if x.next_bb is not None: 
        tr_iloop_ids = [y.id for y in x.next_bb]
        for z in tr_iloop_ids:
            tmp[z].append(id_stem)
for id_iloop, bl_stem_ids in tmp.items():
    df_iloop_cleanup.loc[df_iloop_cleanup['id_bb'] == id_iloop, 'whitelist2'] = df_iloop_cleanup.loc[df_iloop_cleanup['id_bb'] == id_iloop, 'whitelist2'].apply(lambda x: bl_stem_ids)

    
# hloop: whitelist
df_hloop_cleanup['whitelist1'] = None
# df_hloop_cleanup['whitelist1'] = df_hloop_cleanup['whitelist1'].astype('object')
for x in hloop_os_chain:
    id_hloop = x.bb.id
    if x.next_bb is not None:
        tr_stem_ids = [y.id for y in x.next_bb]
        df_hloop_cleanup.loc[df_hloop_cleanup['id_bb'] == id_hloop, 'whitelist1'] = df_hloop_cleanup.loc[df_hloop_cleanup['id_bb'] == id_hloop, 'whitelist1'].apply(lambda x: tr_stem_ids)


In [21]:
pred = predictor.predict(df_stem, df_iloop_cleanup, df_hloop_cleanup)
pred = pred[0, :, 0].detach().numpy()

In [22]:
assert len(pred) == len(df_stem) + len(df_iloop_cleanup) + len(df_hloop_cleanup)

In [23]:
df_stem['pred'] = pred[:len(df_stem)]
df_iloop_cleanup['pred'] = pred[len(df_stem):len(df_stem) + len(df_iloop_cleanup)]
df_hloop_cleanup['pred'] = pred[-len(df_hloop_cleanup):]

In [24]:
df_pred = pd.concat([df_stem, df_iloop_cleanup, df_hloop_cleanup])

In [25]:
df_pred.drop(columns=['num_compatible_stem_outer', 'num_compatible_stem_inner'])

,bb_x,bb_y,siz_x,siz_y,prob_median,n_proposal_norm,bl_x,bl_y,id_bb,blacklist,pred,whitelist1,whitelist2
0,0.0,12.0,4.0,4.0,0.706335,1.000000,3.0,9.0,stem_0,"[stem_0, stem_1, stem_2, stem_3, stem_4, stem_...",0.003859,NaN,NaN
1,0.0,27.0,4.0,4.0,0.295496,1.000000,3.0,24.0,stem_1,"[stem_0, stem_1, stem_2, stem_3, stem_4, stem_10]",0.000427,NaN,NaN
2,0.0,47.0,4.0,4.0,0.201113,0.625000,3.0,44.0,stem_2,"[stem_0, stem_1, stem_2, stem_3, stem_4, stem_...",0.000632,NaN,NaN
3,0.0,54.0,7.0,7.0,0.118634,0.102041,6.0,48.0,stem_3,"[stem_0, stem_1, stem_2, stem_3, stem_4, stem_...",0.001107,NaN,NaN
4,0.0,70.0,8.0,8.0,0.838475,1.000000,7.0,63.0,stem_4,"[stem_0, stem_1, stem_2, stem_3, stem_4, stem_...",0.884917,NaN,NaN
5,4.0,17.0,4.0,4.0,0.137895,0.250000,7.0,14.0,stem_5,"[stem_3, stem_4, stem_5, stem_6, stem_8, stem_9]",0.000510,NaN,NaN
6,8.0,53.0,7.0,7.0,0.692315,1.000000,14.0,47.0,stem_6,"[stem_0, stem_2, stem_3, stem_5, stem_6, stem_...",0.984159,NaN,NaN
7,9.0,23.0,4.0,4.0,0.103329,0.187500,12.0,20.0,stem_7,"[stem_0, stem_6, stem_7, stem_9]",0.000213,NaN,NaN
8,16.0,46.0,4.0,4.0,0.865032,1.000000,19.0,43.0,stem_8,"[stem_2, stem_5, stem_8, stem_9, stem_11, stem...",0.990926,NaN,NaN
9,17.0,34.0,6.0,6.0,0.137925,0.250000,22.0,29.0,stem_9,"[stem_5, stem_7, stem_8, stem_9, stem_10]",0.000664,NaN,NaN


In [34]:
def validate_whitelist(id_bb, picked, remaining, df_info):
    # checks
    assert isinstance(picked, set)
    assert isinstance(remaining, set)
    assert len(picked.intersection(remaining)) == 0
    assert isinstance(df_info, pd.DataFrame)
    
    picked_row = df_info[df_info['id_bb'] == id_bb]
    assert len(picked_row) == 1
    picked_row = picked_row.iloc[0]
    
    # check its whitelist is either in remaining, or have already been picked
    if isinstance(picked_row['whitelist1'], list):
        if not set(picked_row['whitelist1']).intersection(remaining.union(picked)):
            return False
    if isinstance(picked_row['whitelist2'], list):
        if not set(picked_row['whitelist2']).intersection(remaining.union(picked)):
            return False

    return True
        

def pick_one_bb(to_be_picked, picked, remaining, df_info):
    # to_be_picked: set of str, bb IDs
    # picked: set of str, bb IDs
    # remaining: set of str, bb IDs
    # df_info: df with info for each bb, also has whitelist(s) and blacklist
    
    
    # checks
    assert isinstance(to_be_picked, set)
    assert isinstance(picked, set)
    assert isinstance(remaining, set)
    assert to_be_picked.issubset(remaining)
    assert len(to_be_picked.intersection(picked)) == 0
    assert isinstance(df_info, pd.DataFrame)
    
    # subset to_be_picked to be those still in remaining
    to_be_picked = to_be_picked.intersection(remaining)
    
    # subset to to_be_picked
    df_bb = df_info[df_info['id_bb'].isin(to_be_picked)]
    
    # sort by prediction
    df_bb = df_bb.sort_values(by=['pred'], ascending=False)
    
    # keep going until one is selected
    while True:
        # pick the first bb
        picked_row = df_bb.iloc[0]
        id_bb = picked_row['id_bb']

        if validate_whitelist(id_bb, picked, remaining, df_info):
            # return this bb
            return id_bb
        else:  # continue
            # TODO check there's > 1 element left?
            df_bb = df_bb[1:]
    
    # should never happen in practise?
    return None


def add_bb(id_bb, picked, remaining, df_info):
    # add bb, remove blacklist bb from remaining, add bb from whitelist
#     print('debug', id_bb)
#     print('debug', remaining)
    
    # checks
    assert id_bb in remaining
    
    # get info
    picked_row = df_info[df_info['id_bb'] == id_bb]
    assert len(picked_row) == 1
    picked_row = picked_row.iloc[0]
    
    # add bb
    picked.add(id_bb)
    remaining.remove(id_bb)
    print(id_bb)
    print('picked: ', picked)
    print('remaining: ', remaining)
    
    # blacklist
    if isinstance(picked_row['blacklist'], list):
        remaining -= set(picked_row['blacklist'])
    
    # whitelist(s)
    if isinstance(picked_row['whitelist1'], list):
        wl = set(picked_row['whitelist1'])
        if len(wl.intersection(picked)) == 0:
            id_wl1 = pick_one_bb(wl, picked, remaining, df_tmp)
            # need to call add_bb ?! TODO
            picked, remaining = add_bb(id_wl1, picked, remaining, df_info)
    
    if isinstance(picked_row['whitelist2'], list):
        wl = set(picked_row['whitelist2'])
        if len(wl.intersection(picked)) == 0:
            id_wl2 = pick_one_bb(wl, picked, remaining, df_tmp)
            # need to call add_bb ?! TODO
            picked, remaining = add_bb(id_wl2, picked, remaining, df_info)
    
    return picked, remaining

In [35]:
df_tmp = df_pred.copy()
picked = set()
remaining = set(df_tmp['id_bb'])

# sort by pred
df_tmp = df_tmp.sort_values(by=['pred'], ascending=False)

while len(remaining) > 0:  # FIXME debug - replace with a more meaningful condition
    # pick bb with max pred
    df_remaining = df_tmp[df_tmp['id_bb'].isin(remaining)]
    id_bb = df_remaining.iloc[0]['id_bb']
    print('main loop: ', id_bb)
    
    # verify that adding it won't violate whitelist constraint
    if validate_whitelist(id_bb, picked, remaining, df_tmp):
        # add this bb (also takes care of blacklist and whitelist)
        picked, remaining = add_bb(id_bb, picked, remaining, df_tmp)
    else: # otherwise terminate
        break

main loop:  stem_10
stem_10
picked:  {'stem_10'}
remaining:  {'hloop_0', 'stem_8', 'stem_11', 'iloop_2', 'stem_13', 'iloop_0', 'iloop_3', 'hloop_2', 'hloop_1', 'stem_2', 'stem_14', 'stem_6', 'stem_4', 'stem_12', 'stem_0', 'stem_1', 'iloop_6', 'hloop_3', 'stem_3', 'iloop_5', 'stem_7', 'iloop_1', 'stem_5', 'iloop_4', 'hloop_4', 'stem_9', 'hloop_5'}
main loop:  hloop_4
hloop_4
picked:  {'stem_10', 'hloop_4'}
remaining:  {'hloop_0', 'stem_8', 'iloop_2', 'stem_13', 'iloop_0', 'iloop_3', 'hloop_2', 'hloop_1', 'stem_2', 'stem_14', 'stem_6', 'stem_4', 'stem_12', 'stem_0', 'iloop_6', 'hloop_3', 'stem_3', 'iloop_5', 'stem_7', 'iloop_1', 'stem_5', 'iloop_4', 'hloop_5'}
main loop:  iloop_3
iloop_3
picked:  {'stem_10', 'iloop_3', 'hloop_4'}
remaining:  {'hloop_0', 'stem_8', 'iloop_2', 'stem_13', 'iloop_0', 'hloop_2', 'hloop_1', 'stem_2', 'stem_14', 'stem_6', 'stem_4', 'stem_12', 'stem_0', 'iloop_6', 'hloop_3', 'stem_3', 'iloop_5', 'stem_7', 'iloop_1', 'stem_5', 'iloop_4', 'hloop_5'}
stem_8
picked: 

In [36]:
picked

{'hloop_4',
 'iloop_1',
 'iloop_2',
 'iloop_3',
 'stem_10',
 'stem_4',
 'stem_6',
 'stem_8'}

In [37]:
# # a more efficient way to implement this is to maintain a queues
# # queue contains all bbs, sorted according to prediction to start with
# # whenever we pick a bb, or encounter blacklist, items are removed from the queue
# # whenever we encounter whitelist, queue is re-ordered


# df_tmp = df_pred.copy()
# picked = []

# while len(df_tmp) > 0:   # FIXME debug - replace with a more meaningful condition
#     # pick bb with max pred
#     picked_row = df_tmp.loc[df_tmp['pred'] == df_tmp['pred'].max()].iloc[0]
#     id_bb = picked_row['id_bb']
    
#     # before adding in this bb, do one check:
#     # break if none of the whitelist bbs are in the remaining df or list of picked
#     # FIXME this is very greedy! should remove this top pred bb and try the next one (implement a proper queue)
#     if isinstance(picked_row['whitelist1'], list):
#         if not set(picked_row['whitelist1']).intersection(set(df_tmp['id_bb']).union(set(picked))):
#             print("No bb left for {} whitelist1 {}. Terminate.".format(id_bb, picked_row['whitelist1']))
#             break
#     if isinstance(picked_row['whitelist2'], list):
#         if not set(picked_row['whitelist2']).intersection(set(df_tmp['id_bb']).union(set(picked))):
#             print("No bb left for {} whitelist1 {}. Terminate.".format(id_bb, picked_row['whitelist2']))
#             break
    
#     # add this bb
#     picked.append(id_bb)
#     # remove this bb
#     df_tmp = df_tmp[df_tmp['id_bb'] != id_bb]
#     # if blacklist is non-empty, remove those bbs
#     if isinstance(picked_row['blacklist'], list):
#         df_tmp = df_tmp[~df_tmp['id_bb'].isin(picked_row['blacklist'])]    
    
#     print('picked: ', picked)
#     print('left: ', df_tmp['id_bb'].tolist())
    
#     # white list 1
#     if isinstance(picked_row['whitelist1'], list):
#         # ignore this if any whitelist bb has already been picked
#         if set(picked_row['whitelist1']).issubset(set(picked)):
#             pass
#         else:
#             # pick the one with max pred FIXME this might not work! its whitelist might not contain any remaining ones
#             picked_row_1 = df_tmp.loc[df_tmp['pred'] == df_tmp[df_tmp['id_bb'].isin(picked_row['whitelist1'])]['pred'].max()].iloc[0]
#             id_bb_1 = picked_row_1['id_bb']
#             picked.append(id_bb_1)
#             # remove these bbs
#             df_tmp = df_tmp[~df_tmp['id_bb'].isin(picked_row['whitelist1'])]
#             # if blacklist is non-empty, remove those bbs
#             if isinstance(picked_row_1['blacklist'], list):
#                 df_tmp = df_tmp[~df_tmp['id_bb'].isin(picked_row_1['blacklist'])]

#             print('picked: ', picked)
#             print('left: ', df_tmp['id_bb'].tolist())
    
#     # white list 2
#     if isinstance(picked_row['whitelist2'], list):
#         # ignore this if any whitelist bb has already been picked
#         if set(picked_row['whitelist2']).issubset(set(picked)):
#             pass
#         else:
#             # pick the one with max pred
#             picked_row_1 = df_tmp.loc[df_tmp['pred'] == df_tmp[df_tmp['id_bb'].isin(picked_row['whitelist2'])]['pred'].max()].iloc[0]
#             id_bb_1 = picked_row_1['id_bb']
#             picked.append(id_bb_1)
#             # remove these bbs
#             df_tmp = df_tmp[~df_tmp['id_bb'].isin(picked_row['whitelist2'])]
#             # if blacklist is non-empty, remove those bbs
#             if isinstance(picked_row_1['blacklist'], list):
#                 df_tmp = df_tmp[~df_tmp['id_bb'].isin(picked_row_1['blacklist'])]

#             print('picked: ', picked)
#             print('left: ', df_tmp['id_bb'].tolist())

# #     break
    
#     # define back trace point: any combination'ending' with stem (since no whitelist needs to be satisfied)
#     # on the other hand, we might not realize that some iloop/hloop cannot be included until we check their
#     # whitelist and realize that none of the compatible bbs are there
    
    

In [ ]:
# picked

In [38]:
df_pred[df_pred['id_bb'].isin(picked)][['bb_x', 'bb_y', 'siz_x', 'siz_y', 'pred', 'id_bb']]

,bb_x,bb_y,siz_x,siz_y,pred,id_bb
4,0.0,70.0,8.0,8.0,0.884917,stem_4
6,8.0,53.0,7.0,7.0,0.984159,stem_6
8,16.0,46.0,4.0,4.0,0.990926,stem_8
10,25.0,41.0,7.0,7.0,0.997029,stem_10
4,7.0,63.0,2.0,11.0,0.990775,iloop_1
10,14.0,47.0,3.0,2.0,0.984288,iloop_2
11,19.0,43.0,7.0,3.0,0.993706,iloop_3
4,31.0,35.0,5.0,5.0,0.996665,hloop_4


In [39]:
pd.DataFrame(data['df_target'])

,bb_x,bb_y,siz_x,siz_y,bb_type
0,0,70,7,7,stem
1,9,23,4,4,stem
2,25,41,5,5,stem
3,47,63,5,5,stem
4,12,20,9,9,hloop
5,29,37,9,9,hloop
6,51,59,9,9,hloop
